# Pits, SMP, and SSA Oh My 

The notebook attempts to use the database to find all the SMP profiles associated with Pits where SSA was also measured 

We have micro-ct data from the pits:

* 1S17  
* 2N13
* 2S16 
* 2S7 
* 9C16

We can use these to determine which of the SMP profile file names should be downloaded. We are currently only focused on the smp profiles nearest the pit

In [ ]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# Our Integrating Sphere SSA and SMP data in the DB are under layers
from snowexsql.data import LayerData, SiteData

# import our func to access postgis functions
from sqlalchemy.sql import func 

# Grab our handy functions to convert records to dataframes
from snowexsql.conversions import query_to_pandas, query_to_geopandas
from datetime import date 

# Import plotting tools 
import matplotlib.pyplot as plt 

# Use pandas for concatting dataframes 
import pandas as pd 
import geopandas as gpd 

from geoalchemy2.shape import from_shape, to_shape

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@52.32.183.144/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name, credentials=None)

# Sites where Micro-ct was collected
sites = ['1S17', '2N13', '2S16', '2S7','9C16']

### Dates and Locations

The following block queries the database for dates that the SMP was used at the pits of interest. It then grabs those locations to see where all the profiles are. We expect these to be in some sort of cross pattern per the sampling strategy. So this plot is more for confirmation we are grabbing the right stuff. 


In [ ]:
# Determine what dates all the instruments were recorded at these pits 
smp_dates = {}

session.rollback()

# Grab the sitedata geometry for plotting 
qry = session.query(SiteData.geom, SiteData.site_id).filter(SiteData.site_id.in_(sites)).distinct()
site_df = query_to_geopandas(qry, engine)

# make a subplot of len == len of sites
fig, axes = plt.subplots(1, len(sites), figsize=(11,8))

for i, sid in enumerate(sites):
            
    # Filter the layer data by site_id Use .contains() here since there is some inconsistency in site_id naming convention. E.g. 1S17 vs COGM1S17_20200128
    qry = session.query(LayerData.site_id, LayerData.geom, LayerData.date).filter(LayerData.site_id.contains(sid))

    # Filter the layer data smp profiles
    qry = qry.filter(LayerData.instrument == 'snowmicropen')
    
    # Unique dates only 
    qry = qry.distinct()
    
    # Execute the query and convert to  geodataframe
    df = query_to_geopandas(qry, engine)
    
    # Plot all the SMP locations
    df.plot(ax=axes[i], color='black', marker='2')
    
    # Plot the pit location
    ind = site_df['site_id'] == sid
    site_df[ind].plot(ax=axes[i], color='magenta', marker='^')
    smp_dates[sid] = df['date'].unique()
    print(f'Found {len(smp_dates[sid])} dates where SMP was used at Site {sid}')
                                                        

In [ ]:
session.rollback()
def get_smp_within_pit_radius(site_id, radius, session, engine):
    """
    Grabs smp location and comments within a radius in meters of a know site
    
    Args:
        site_id: String to search for sites ids
        radius: Distance in meters to search for smp profiles from the pit 
        session: Session object from the get_db function 
        engine: Engine object from the get_db_function
    
    Returns:
        df: Geopandas dataframe containing unique SMP locations and their comments
    """
    # Grab raw geom object (location) for this site
    geom = session.query(SiteData.geom).filter(SiteData.site_id == site_id).distinct().all()[0][0]
    
    # Build a circle on the location 
    circle = session.query(func.ST_Buffer(geom, radius)).all()[0][0] # To use with the db 
    circle_s = gpd.GeoSeries(to_shape(circle))                      # For plotting
        
    # The original filename is stored in the comments of the smp profile
    qry = session.query(LayerData.geom, LayerData.comments)
    
    # Filter on our micropen
    qry = qry.filter(LayerData.instrument == 'snowmicropen')
    
    # Filter on the data within the buffer 
    qry = qry.filter(func.ST_Within(LayerData.geom, circle))
    
    # Get the unique values (layer have all this data for each layer)
    qry = qry.distinct()
    
    # Make a sweet geodataframe
    df = query_to_geopandas(qry, engine)
    
    return df, circle_s


# Loop over the sites 
search_radius = 10 
for i, site_id in enumerate(sites):

    df,  circle_s = get_smp_within_pit_radius(site_id, search_radius, session, engine)
    
    # plot our circle
    ax = circle_s.plot(color='blue', edgecolor='black', alpha=0.25)
    
    # Plot profile locations
    df.plot(ax=ax)
    
    # plot the site for reference
    ind = site_df['site_id'] == site_id
    site_df[ind].plot(ax=ax, color='magenta', marker='^') 
    
    ax.set_title(f'SMP locations for Site {site_id} witihin {search_radius} meters')
    ax.ticklabel_format(style='plain', useOffset=False)
    
    print(', '.join([v.split(',')[0].replace('fname = ','') for v in df['comments'].values]))
    print('\n')

In [ ]:
session.close()